In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

# 加载我们自定义的工具函数
from testCases import *
from dnn_utils import *

# 设置一些画图相关的参数
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

In [51]:
# 该函数用于初始化所有层的参数w和b
def initialize_parameters_deep(layer_dims):
    """
    参数:
    layer_dims -- 这个list列表里面，包含了每层的神经元个数。
    例如，layer_dims=[5,4,3]，表示输入层有5个神经元，隐藏层第一层有4个，隐藏层第二层有3个神经元
    
    返回值:
    parameters -- 这个字典里面包含了每层对应的已经初始化了的W和b。
    例如，parameters['W1']装载了第一层的w，parameters['b1']装载了第一层的b
    """
    L = len(layer_dims)  # 一共有多少层
    parameters = {}
    np.random.seed(1)
    
    # 遍历每一层，为每一层的W和b进行初始化
    for i in range(1, L):  # layer_dims[0] 表示输入有多少个神经元, 只需要初始化隐藏层的参数
        # 维度部分可参考笔记: 第i层Wi的维度是(n[i] , n[i-1])
#         parameters['W' + str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1]) / np.sqrt(layer_dims[i-1])  # 一行是一个神经元对每个输入的权重, 缩小的程度应该和每个神经元有多少个权重有关, 所以是除以np.sqrt(layer_dims[i-1]), 而不是np.sqrt(layer_dims[i])
        parameters['W' + str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1])  # test
        parameters['b' + str(i)] = np.zeros((layer_dims[i], 1))
        
        # 核对一下W和b的维度是我们预期的维度
        assert(parameters['W' + str(i)].shape == (layer_dims[i], layer_dims[i - 1]))
        assert(parameters['b' + str(i)].shape == (layer_dims[i], 1))
    
    return parameters

In [52]:
# 看看效果
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.72642933 -0.27358579 -0.23620559 -0.47984616  0.38702206]
 [-1.0292794   0.78030354 -0.34042208  0.14267862 -0.11152182]
 [ 0.65387455 -0.92132293 -0.14418936 -0.17175433  0.50703711]
 [-0.49188633 -0.07711224 -0.39259022  0.01887856  0.26064289]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.55030959  0.57236185  0.45079536  0.25124717]
 [ 0.45042797 -0.34186393 -0.06144511 -0.46788472]
 [-0.13394404  0.26517773 -0.34583038 -0.19837676]]
b2 = [[0.]
 [0.]
 [0.]]


In [53]:
# 前向传播, Z^[i] = W^[i]A^[i-1]+b^[i]
def linear_forward(A, W, b):   
    Z = np.dot(W, A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b) # 将这些变量保存起来，因为后面进行反向传播时会用到它们
    
    return Z, cache

In [54]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


In [57]:
# 实现公式 A[i]=g(Z[i])，g代表激活函数，使上面的线性前向传播就变成非线性前向传播。激活函数sigmoid和relu定义在dnn_utils.py中
def linear_activation_forward(A_prev, W, b, activation):
    """
    Arguments:
    A_prev -- 上一层得到的A，输入到本层来计算Z和本层的A。第一层时A_prev就是特征输入X
    W -- 本层相关的W
    b -- 本层相关的b
    activation -- 两个字符串，"sigmoid"或"relu"，指示该层应该使用哪种激活函数
    """
    Z, linear_cache = linear_forward(A_prev, W, b)
    if activation == "sigmoid":
        A = sigmoid(Z)
    elif activation == "relu":
        A = relu(Z)
    else:
        raise ValueError
        
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
#     cache = (linear_cache, Z) # 缓存一些变量，后面的反向传播会用到它们
        
    return A

In [59]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [ ]:
# 这个函数构建了一个完整的前向传播过程。这个前向传播一共有L层，前面的L-1层用的激活函数是relu，最后一层使用sigmoid。
def L_model_forward(X, parameters):
    """
    参数:
    X -- 输入的特征数据
    parameters -- 这个list列表里面包含了每一层的参数w和b
    """
    
    